## Lunar Lander con Deep Q Learning

## Integrantes

- Juan Jose Urioste
- Carlos Huerta Garcia
- Alejo Torres Teruel

## Instalacion

Todas las dependencias necesarias estan listadas en el fichero `requirements.txt`. Que pueden ser instaladas con un manejador de paquetes de python como `pip` o `conda`.

## DevContainer

Ademas se provee una descripcion del Entorno de desarrollo utilizado para trabajar con el entorno de gymnasium.

como comando de post creacion se especifica la instalacion y compilacion de las dependencias necesarias para el entorno de gymnasium y el proyecto.


## Importamos las librerias necesarias



In [1]:
import random
import gymnasium as gym
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
from collections import deque


2023-05-05 08:34:01.826484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Agente de DQL

la clase DQLAgent se encargara de manejar la logica de entrenamiento y evaluacion del agente.
Va a depender del entorno sobre el cual se ejecute y agrupa los metodos y los atributos necesarios para implementar el algoritmo de DQL.Mas en especifico, en este caso se trata de DDQL ya que contamos con un modelo de target y otro de entrenamiento.


## Atributos

- `state_size`: dimension del espacio de estados
- `action_size`: dimension del espacio de acciones

Ambos son dependientes de entorno sobre el cual se ejecute el agente.

- `epsilon`: factor de exploracion
- `epsilon_decay`: factor de decaimiento de epsilon -> este factor se utiliza para actualziar el valor de epsilon en cada episodio esto con el fin de que el agente explore menos y explote mas a medida que avanza el entrenamiento
- `epsilon_min`: valor minimo de epsilon
- `learning_rate`: factor de aprendizaje -> a fin de cuenta nuestros modelos son modelos de Deep learning en este caso perceptrones multicapas con capas full conected por lo que el aprendizaje se realiza mediante el algoritmo de SGD y este factor es el que determina la magnitud de los cambios en los pesos de las conexiones de las neuronas
- `memory`: Es una estructura de datos optimizada para accesar los datos de sus extremos, en este caso se utiliza para almacenar las transiciones de la forma (estado, accion, recompensa, estado siguiente, done) y poder muestrear de forma aleatoria hacer que el modelo "reviva" experiencias pasadas y pueda aprender de ellas
- `model`: modelo de entrenamiento sobre el cual se va a realizar el aprendizaje
- `target_model`: modelo de target que se actualiza cada cierta cantidad de episodios y es aquel que se utiliza para predecir el valor de Q en el estado siguiente


## Metodos

- `build_model`: construye el modelo de Deep learning que se va a utilizar para entrenar el agente
- `remember`: almacena una transicion en la memoria del agente
- `act`: retorna la accion a tomar en el estado actual infiriendola en base al modelo
- `replay`: muestrea la memoria de manera aleatoria y crea un batch usando aquellas memorias que no terminaron el episodio, predecimos usando ambos modelos y sobre las memorias donde no termino el episodio actualizamos los valores Q de acuerdo a la formula de DQL, luego recuperamos las acciones desde el batch al igual que los targets que se actualizaran con el valor calculado anteriormente para luego entrenar el modelo. vamos a entrenar en un epoch ya que este procedimiento sucede en cada en cada step de cada episodio
- `adaptiveEGreedy`: Nos permite ajustar el factor de exploracion
- `targetModelUpdate`: Actualizamos los pesos del modelo de target con los pesos del modelo de entrenamiento. Ambos cuentan con la misma arquitectura


In [2]:
class DQLAgent:
    def __init__(self, env):
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.memory = deque(maxlen=4000)
        self.model = self.build_model()
        self.target_model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, s):
        if np.random.rand() <= self.epsilon: #! epsilon greedy
            return np.random.choice(self.action_size)
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def replay(self,batch_size):
        if len(agent.memory) < batch_size: #! no tengo suficientes memorias
            return

        minibatch = random.sample(self.memory, batch_size) #! samplear la memoria tomando como tamaño batch_size
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False) #! me interesa saber los que no termino 
        y = np.copy(minibatch[:, 2])

        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            y[not_done_indices] += np.multiply(self.gamma, predict_sprime_target[not_done_indices, np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(batch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=1, verbose=0)
        

            
    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def targetModelUpdate(self):
        self.target_model.set_weights(self.model.get_weights())
        

# Entrenamiento

- Creamos el entorno e instanciamos el agente
- definimos el tamaño del batch y la cantidad de episodios a entrenar
- Por cada episodio vamos a resetear el entorno y el estado del agente
- En cada step vamos a pedir una accion inferida por el a gente para ejecutarse en el entorno, luego memorizar la transicion y realizar el proceso de replay

- si el episodio finaliza vamos a actualizar el modelo de target

In [ ]:
env = gym.make('LunarLander-v2')

state, _ = env.reset()

agent = DQLAgent(env)
state_number = env.observation_space.shape[0]

batch_size = 32
episodes = 5
for e in range(episodes):

    state, _ = env.reset()
    state = np.reshape(state, [1, state_number])
    total_reward = 0
    for time in range(1000):

        action = agent.act(state)

        next_state, reward, done, _, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_number])

        agent.remember(state, action, reward, next_state, done)

        state = next_state

        agent.replay(batch_size)

        total_reward += reward

        if done:
            agent.targetModelUpdate()
            break

    agent.adaptiveEGreedy()

    print('Episode: {}, Reward: {}'.format(e, total_reward))


# Probamos el modelo
ejecutamos un bucle PPA pero con un entorno en modo de renderizado humano para poder visualizar el comportamiento del agente

In [ ]:
env = gym.make('LunarLander-v2', render_mode='human')

state, _ = env.reset()

terminated = truncated = False
while not (terminated or truncated):
    action = agent.act(state)
    state, _, terminated, truncated, _ = env.step(action)
env.close()